Embeddings are vectors in higher dimension. Same vectors or same no.s will have same meanings or nearly same meanings. There are 2 ways to create embeddings: 1. word2vec 2. Keras Embeddings

Assumption of word2vec is the meaning of the word changes depending on other words in a sentence. For eg Data in I am a Data scientist, Data Analyst, Data Engineer or war in Civil War, Cold War.

There are 2 types of algorithms in word2vec 1. Continuous Bag of Words 2. skip gram. There are some hyperparameters which are very important like windowsize in CBOW

In [ ]:
# !pip install keras

In [ ]:
# !pip install keras-preprocessing

In [ ]:
# pip install --upgrade tensorflow keras

In [ ]:
import re
import numpy as np
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Flatten,Dropout,BatchNormalization

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

KeyboardInterrupt: 

In [ ]:
import pandas as pd

# URL of the CSV file
url = "https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv"

# Read the CSV into a DataFrame
df = pd.read_csv(url, encoding='latin-1')  # encoding='latin-1' helps avoid UnicodeDecodeError

# Display the first few rows
print(df.head())


In [ ]:
df.head()

In [ ]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
df.head()

In [ ]:
df.columns=['label','text']

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df['label'].value_counts()

In [ ]:
df['label'].value_counts().plot(kind="bar")

In [ ]:
# Text preprocessing -- tokenization, stopwords and punctuations removal,
# removing non-alphabets, lemmatization (preferred)

In [ ]:
def preprocessing_pipeline(text):
  lemmatizer=WordNetLemmatizer()
  if str(type(text)) != "<class 'str'>":
    text=list(text)
    corpus=[]
    for i in range(len(text)):
      review=re.sub('[^a-zA-Z]',' ',text[i])
      review=review.lower()
      review=review.split()
      r=[word for word in review if not word in stopwords.words('english')]
      r=[lemmatizer.lemmatize(word) for word in r]
      r=' '.join(r)
      corpus.append(r)
    return corpus
  else:
    review=re.sub('[^a-zA-Z]',' ',text[i])
    review=review.lower()
    review=review.split()
    r=[word for word in review if not word in stopwords.words('english')]
    r=[lemmatizer.lemmatize(word) for word in r]
    r=' '.join(r)
    return r

In [ ]:
preprocessed_text=preprocessing_pipeline(df['text'])
print(preprocessed_text)

In [ ]:
preprocessing_pipeline(["Go until jurong point, crazy.. Available only"])

In [ ]:
df['label']=np.where(df['label']=='ham',0,1)
df.head()

In [ ]:
X=df['text']
y=df['label']
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

In [ ]:
# Featute Extraction (bagofwords-countvectorizer OR Tfidf)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
xtrain_cv = cv.fit_transform(xtrain)
xtest_cv = cv.transform(xtest)

print(xtrain_cv)
print(xtrain_cv.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
xtrain_tf = tf.fit_transform(xtrain)
xtest_tf = tf.transform(xtest)

print(xtrain_tf)
print(xtrain_tf.shape)

In [ ]:
# !pip install gensim

Gensim is a powerful and efficient python library for NLP. Its popular for working with large text corpora, word embeddings and unsupervised text analysis. It works well with nltk, Spacy and scikitlearn.

In [ ]:
from gensim.models import Word2Vec,KeyedVectors
sentences=[sentence.split() for sentence in df['text']]
w2v_model=Word2Vec(sentences,vector_size=300,workers=4,min_count=5,window=10)
# hyperparameters -- VectorSize is your column size

In [ ]:
#generating embeddings ---
def vectorize(sentence):
  words=sentence.split()
  words_vectors=[w2v_model.wv[word] for word in words if word in w2v_model.wv]
  if len(words_vectors)==0:
    return np.zeros(300)
  word_vecs=np.array(words_vectors)
  return word_vecs.mean(axis=0)
xtrain_embed=np.array([vectorize(sentence) for sentence in xtrain])
xtest_embed=np.array([vectorize(sentence) for sentence in xtest])

In [ ]:
xtest_embed.shape

In [ ]:
xtrain_embed.shape

In [ ]:
# pretrained embeddings download
import gensim.downloader as api
model=api.load('word2vec-google-news-300')


In [ ]:
print(model['man'].shape)
print(model['man']) #embedding
print(model.most_similar('cricket'))
print(model.similarity('bat','ball'))
print(model.doesnt_match(['PHP','java','monkey']))
vec=model['king']-model['man']+model['woman']
model.most_similar([vec])

In [ ]:
# This u have done using CountVectorizer or bagofwords
lr=LogisticRegression()
lr.fit(xtrain_cv,ytrain)
ypred=lr.predict(xtest_cv)
from sklearn.metrics import confusion_matrix, classification_report
d=pd.DataFrame(confusion_matrix(ytest,ypred),index=['ham','spam'],columns=['ham','spam'])
print(d)
print(classification_report(ytest,ypred))

In [ ]:
# This u have done using Tfidf

lr = LogisticRegression()
lr.fit(xtrain_tf, ytrain)
ypred = lr.predict(xtest_tf)
from sklearn.metrics import confusion_matrix, classification_report
d=pd.DataFrame(confusion_matrix(ytest,ypred),index=['ham','spam'],columns=['ham','spam'])
print(d)
print(classification_report(ytest,ypred))

In [ ]:
# This u have done using Gensim Model Word2Vec
lr=LogisticRegression()
lr.fit(xtrain_embed,ytrain)
ypred=lr.predict(xtest_embed)
from sklearn.metrics import confusion_matrix, classification_report
d=pd.DataFrame(confusion_matrix(ytest,ypred),index=['ham','spam'],columns=['ham','spam'])
print(d)
print(classification_report(ytest,ypred))

In [ ]:
# pretrained embeddings download
import gensim.downloader as api
word_vectors=api.load('word2vec-google-news-300')

#generating embeddings ---
def vectorize(sentence):
  words=sentence.split()
  words_vectors=[word_vectors[word] for word in words if word in word_vectors]
  if len(words_vectors)==0:
    return np.zeros(300)
  word_vecs=np.array(words_vectors)
  return word_vecs.mean(axis=0)
xtrain_embed_google=np.array([vectorize(sentence) for sentence in xtrain])
xtest_embed_google=np.array([vectorize(sentence) for sentence in xtest])


In [ ]:
# This u have done using Gensim Model google
lr=LogisticRegression()
lr.fit(xtrain_embed_google,ytrain)
ypred=lr.predict(xtest_embed_google)
from sklearn.metrics import confusion_matrix, classification_report
d=pd.DataFrame(confusion_matrix(ytest,ypred),index=['ham','spam'],columns=['ham','spam'])
print(d)
print(classification_report(ytest,ypred))

##**CREATION OF ANN**

In [ ]:
# usING Bag of Words--

model=Sequential()
model.add(Dense(64,input_shape=(xtrain_cv.shape[1],),activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(16,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())

In [ ]:
model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(xtrain_cv.toarray(),ytrain,epochs=20,batch_size=4,validation_split=0.2)

In [ ]:
p=model.predict(xtest_cv.toarray())
ypred=np.where(p>0.5,1,0)

In [ ]:
d=pd.DataFrame(confusion_matrix(ytest,ypred),index=['ham','spam'],columns=['ham','spam'])
print(d)
print(classification_report(ytest,ypred))

**USING EMBEDDINGS IN KERAS LIBRARY**

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(xtrain)
xtrain=tokenizer.texts_to_sequences(xtrain)
x_test=tokenizer.texts_to_sequences(xtest)


In [ ]:
vocab_size=len(tokenizer.word_index)+1  #tokenizer.word_index is a dictionary that maps ech unique word to
# an integer index
print(vocab_size)

In [ ]:
max_length=100
x_train=pad_sequences(x_train,maxlen=max_length,padding='post')
x_test=pad_sequences(x_train,maxlen=max_length,padding='post')


In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=32,input_length=max_length))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(16,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
print(model.summary())

In [ ]:
p=model.predict(x_test)
ypred=np.where(p>0.5,1,0

SyntaxError: incomplete input (ipython-input-1-599812715.py, line 2)

In [ ]:
d=pd.DataFrame(confusion_matrix(ytest,ypred),index=['ham','spam'],columns=['ham','spam'])
print(d)
print(classification_report(ytest,ypred))

NameError: name 'pd' is not defined